# OFDM Receiver Tutorial

Authors: Meles Gebreyesus Weldegebriel and Neal Patwari
Version: Feb 2026. Version 0.3.

This code assumes that someone has done the following before you run this tutorial.
1. Generates an OFDM packet for transmission over SDRs (using UHD drivers for control).
2. The packet can then be transmitted over the air in POWDER using Shout using the given instructions.

In this lab, we're going to run only the receiver.

This code  allows you to:
1. Load the received data from file.
2. Synchronize to and then demodulate the data in the OFDM packet.

For the receiver, we use the stop sign emoji (🛑) to show you particular parts that you will need to implement code in. Look for these as you progress.



In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import scipy.signal as signal
from itertools import chain
from scipy.signal import correlate, correlation_lags, fftconvolve

In [ ]:
# Load in files from the github repo for this tutorial.
user = "npatwari"
repo = "OFDM_RX_Tutorial"

# remove local directory if it already exists
if os.path.isdir(repo):
    !rm -rf {repo}

!git clone https://github.com/{user}/{repo}.git

## 1. Info about the OFDM transmitted signal


This is where the user can edit the OFDM parameters, including the FFT size, the cyclic prefix (CP) size, the number (and locations) of pilot and guard band subcarriers and the pilot value. **This must match the parameters used in the transmitter**. The 2nd code block below is for displaying to the user which subcarriers are assigned to which purpose.

* The `FFT_size` length vector that represents the OFDM symbol in the frequency domain, one value for each subcarrier.
* `data_size` QPSK symbol values that are in the Symbol vector, but in the specific `dataCarriers` indices. One QPSK symbol for each dataCarriers index.
*   `pilotValue` is the constant complex value in every pilot subcarrier, as listed in the `pilotCarriers` vector.
*   You don't need to do anything for the guard band subcarriers, because they should be zero.

In [ ]:
# Basic OFDM and system parameters
FFT_size = 64                       # Number of FFT points
OFDM_size = 80                      # Total size including cyclic prefix
data_size = 48                      # Number of data subcarriers
mess_length = 560                   # Message length in bits
CP = 16                             # Cyclic prefix length
pilotValue = 2* (1.4142 + 1.4142j)  # Pilot IQ value

# Subcarrier number assignments used in the TX & RX
pilotCarriers = np.array([-21,-7,7,21])  # Pilot carriers
dataCarriers = np.array([ -26,-25,-24,-23,-22,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,
                          -9,-8,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,
                          18,19,20,22,23,24,25,26])

# All subcarriers are numbered to center at zero.
allCarriers = np.arange(-FFT_size//2, FFT_size//2)
reserved = np.concatenate((dataCarriers, pilotCarriers))
# guard band subcarriers are whatever is remaining.
guardCarriers = np.array([sc for sc in allCarriers if sc not in reserved])

# Print the count of each type of subcarrier to the
print('Number of carriers:', len(allCarriers))
print('Number of pilot carriers:', len(pilotCarriers))
print('Number of data carriers:', len(dataCarriers))
print('Number of guard carriers:', len(guardCarriers))


In [ ]:
# Plot a (1-D) view of the subcarriers, showing guard bands, pilots, and data subcarriers.
plt.figure()
plt.plot(pilotCarriers, np.zeros_like(pilotCarriers), 'bo', label='pilot subcarriers')
plt.plot(dataCarriers, np.zeros_like(dataCarriers), 'ro', label='data subcarriers')
plt.plot(guardCarriers, np.zeros_like(guardCarriers), 'ko', label='guard subcarriers')
plt.xlabel("Subcarrier #")
plt.gca().set_aspect(aspect=150) # doesn't need to be tall
plt.ylim(-0.01, 0.1)
plt.yticks([]) # avoid the impression we're plotting something on the y-axis
plt.legend()

Our simple OFDM packet structure is as follows:

  | PREAMBLE | DATA |
  | -------- | ------- |
  | 2 symbols | 7 symbols |

The PREAMBLE is two repetitions of the symbol stored in `preamble.mat`. The DATA symbols are created here in the function `generate_ofdm_signal()`.

In [ ]:
# We pull the previously-generated PREAMBLE symbol from file.
# It can be repeated for increased robustness.
# The preamble is called the LTF in 802.11n
def generate_ltf():
    mat = scipy.io.loadmat('./' + repo + '/preamble.mat')
    ltf = mat['ltf'].flatten()  # complex array
    return np.tile(ltf, 1)      # repeat 2 times

## 3. OFDM Receiver

Here, we load the rx signal from a previously conducted OTA experiment file.

🛑 Chose which data file you want. There are three. The medium-SNR measurement is the '10dB_rx_output.dat' file. You can alternatively use the '15dB_rx_output.dat' file to see an even cleaner signal, or the '5dB_rx_output.dat' to have more of a challenge.

In [ ]:
## Load a received IQ file from file.
received_filename =   # You pick one of the ``.dat`` files.
rx_signal = np.fromfile('./' + repo + '/' + received_filename, dtype=np.complex64)

### 3.1 Packet time synchronization

The cross correlation in this notebook is implemented in a different, more computationally efficient manner than we did in the QPSK lab. In the prior lab, we used numpy's `convolve` function, which does the convolution in the time domain. In the time domain, it is a sum of the product of the two signals just for one delay value. Thus this sum of the product must be calculated for each delay. In the frequency domain, the full convolution is computed with a single product. But for the frequency domain convolution method we need to add the computation of the Fourier transforms of both signals, and the inverse Fourier transform of the product. Still, the frequency domain method is faster for our case.

In [ ]:
def cross_correlation_max(rx0, preamble, preamble_scale=0.001):
    """
    Cross-correlate rx0 with a known preamble and plot:
      - RX real/imag with the detected preamble overlaid at the detected start index
      - Correlation magnitude (or NCC) vs. sample index (no negative lags)
    """

    rx0 = np.asarray(rx0, dtype=np.complex64)
    preamble = np.asarray(preamble, dtype=np.complex64)

    # --- Complex matched filter correlation ---
    kernel = np.conj(preamble[::-1])
    xcorr_out = signal.fftconvolve(rx0, kernel, mode='valid')  # valid mode → only non-negative shifts
    metric = np.abs(xcorr_out)

    # Return the index of the peak of the correlation output.
    preamble_start = int(np.argmax(metric))

    print(f"Detected preamble at sample index {preamble_start}")

    # --- Plot the output so the user can validate that it is working ---
    fig, axes = plt.subplots(2, 1, figsize=(11, 6))
    # Top: RX signal with overlaid preamble
    axes[0].plot(np.real(rx0), label='Real RX Signal')
    axes[0].plot(np.imag(rx0), label='Imag RX Signal')
    overlay_x = np.arange(preamble_start, preamble_start + len(preamble))
    overlay_p = preamble * preamble_scale
    axes[0].plot(overlay_x, np.real(overlay_p), label='Preamble (real, scaled)')
    axes[0].plot(overlay_x, np.imag(overlay_p), label='Preamble (imag, scaled)', alpha=0.7)
    axes[0].set_title("Received Signal with Detected Preamble Overlay")
    axes[0].legend(loc='upper right')
    # Bottom: correlation metric vs. index
    axes[1].plot(metric, label="|Cross-correlation|")
    axes[1].axvline(preamble_start, linestyle='--', color='r', label=f'Detected start = {preamble_start}')
    axes[1].set_xlabel("Sample index")
    axes[1].set_ylabel("|Cross-correlation|")
    axes[1].legend(loc='upper right')
    axes[1].set_title(f"Cross-Correlation")
    axes[1].grid(True)
    plt.tight_layout()
    plt.show()

    return preamble_start


Some bit and symbol conversion functions; and our implementation of the symbol detector.

In [ ]:
def binvector2str(bits):
    """
    Convert a flat iterable of 0/1 to a 7-bit ASCII string (MSB first).
    Pure-Python, no NumPy required.
    """
    bits = list(bits)
    if len(bits) % 7 != 0:
        raise ValueError("Length of bit stream must be a multiple of 7.")

    out_chars = []
    for i in range(0, len(bits), 7):
        byte = 0
        # MSB first: positions 0..6 map to weights 64..1
        # converts the binary chunk into its integer value
        for b in bits[i:i+7]:
            byte = (byte << 1) | (1 if b else 0)
        out_chars.append(chr(byte))
    # Joins the list of characters into a single string
    rx_text = ''.join(out_chars)
    print('Received Message:', rx_text)
    return rx_text

def mary2binary(data, M):
    length = len(data) # number of values in data
    log2M = round(np.log2(M)) # integer number of bits per data value
    format_string = '0' + str(log2M) + 'b'
    binarydata = np.zeros((1,length*log2M))
    count = 0
    for each in data:
        binval = format(int(each), format_string)
        for i in range(log2M):
            binarydata[0][count+i] = int(binval[i])
        count = count + log2M
    return binarydata

def findClosestComplex(r_hat, outputVec):
    # outputVec is a 4-length vector for QPSK, would be M for M-QAM or M-PSK.
    # This checks, one symbol sample at a time,  which complex symbol value
    # is closest in the complex plane.
    data_out = [np.argmin(np.abs(r-outputVec)) for r in r_hat]
    return data_out

def constellation_plot(signal):
    plt.figure(figsize=(5,5))
    ax = plt.gca()
    ax.set_aspect(1.0)
    plt.plot(np.real(signal), np.imag(signal),'ro')
    plt.ylabel('Imag(Symbol Sample)', fontsize=14)
    plt.xlabel('Real(Symbol Sample)', fontsize=14)
    plt.title('Constellation Plot')
    plt.grid('on')
    plt.tight_layout()


This is now the main code which calls the above functions.
- Note that in the saved `.dat` receiver files, the files are from real-time experiments on POWDER which start recording the `.dat` file when it finds the start of a packet. So when reading .dat files we *expect* to see a `preamble_start` index of 0.

In [ ]:
## Load the packet preamble and cross-correlate it with the received signal.
preamble = generate_ltf()
preamble_start = cross_correlation_max(rx_signal,preamble)
print("Preamble starts at index", preamble_start)

We can easily estimate the complex (magnitude, angle) gain that each pilot subcarrier gets multiplied by, because we know what the original complex amplitude that was sent in the pilot subcarrier (`pilotValue`). This code calls those pilot subcarrier gains as `Hp`. But full channel estimation requires interpolating between pilot subcarriers to estimate the complex (magnitude, angle) gain that every subcarrier gets multiplied by. This code calls that `H_full`.

🛑 You will implement:
1. Estimating the channel gain by dividing the measured (received) pilot value by the pilot value that was known to be sent.

In [ ]:
def Channel_Estimation(signal):
    """
    Estimate the per-subcarrier channel H[k] for ONE OFDM symbol using pilots only.
    Strategy:
      1. Pull out the complex amplitude in the pilot subcarriers.
      2. Divide by Xp, the known pilot symbol value that was transmitted.
      3. Perform interpolation to estimate the channel at the data subcarriers.
         (Do linear interpolation on the log magnitude and on phase, separately)

    """
    eps = 1e-12  # small number to prevent divide-by-zero / log(0)

    # Pull out the complex amplitude in the pilot subcarriers.
    # Call your output `Yp`
    ### YOUR CODE HERE

    # Divide by Xp, the known pilot symbol value that was transmitted
    # Make sure you don't divide by zero!  Call the output `Hp`
    ### YOUR CODE HERE

    # Perform interpolation to estimate channel at data subcarriers.
    # The literature says it is best to do interpolation separately in:
    # (a) log magnitude,  and (b) phase (unwrapped).

    # Split Hp into magnitude and phase for interpolation -
    mag_p = np.maximum(np.abs(Hp), eps)  # clamp to eps to avoid log(0)
    log_mag_p = np.log(mag_p)
    # Phase unwrap across pilot indices to avoid discontinuities at +/-pi boundaries.
    phase_p = np.unwrap(np.angle(Hp))

    # Interpolate (linear) from pilot bins to ALL ACTIVE subcarriers
    log_mag_all = np.interp(allCarriers, pilotCarriers, log_mag_p)
    phase_all   = np.interp(allCarriers, pilotCarriers, phase_p)

    # Recombine to complex H on active carriers
    H_estimate = np.exp(log_mag_all) * np.exp(1j * phase_all)

    # Pack into a full FFT-sized vector, leaving guards/DC as zeros
    H_full = np.zeros_like(signal, dtype=complex)
    H_full[allCarriers] = H_estimate

    return H_full

Here is where the OFDM receiver operates, symbol by symbol. There is a for loop that operates on each `OFDM_size` samples until the packet is done.

🛑 You will implement:
1. Cyclic prefix removal
2. Computation of the FFT to convert the time-domain signal back into the frequency domain.
3. Using an estimate of the channel gain coefficients (estimated using the pilot subchannels), you will implement the equalization.

You may need to debug your code, so feel free to add print statements or plots. You should probably though put then in an `if i==0:` block so that they don't print out for all symbols.

In [ ]:
# Calculate the start index of the DATA part of the packet. Then copy `mess_length`
# samples, starting from that start index, into `data_signal`.
data_start = preamble_start + len(preamble)
data_signal = rx_signal[data_start: data_start + mess_length]

# Process each symbol (which is `OFDM_size` samples).
# Remember `OFDM_size` is the total symbol length including the part that will
# have an FFT taken, and the cyclic prefix which will be dropped.
for i in range(len(data_signal)//(OFDM_size)):

    # Pick out the next `OFDM_size` of samples from `data_signal`,
    # and call it `data_cp`.
    ### YOUR CODE HERE

    # Remove from data_cp the cyclic prefix, which is the first `CP` samples.
    # Call the output `data_without_cp`.
    ## YOUR CODE HERE ##

    # Take the FFT using np.fft.fft() on the non-CP symbol samples `data_without_cp`.
    # Remember the FFT is length `FFT_size`.
    # Call the output `OFDM_freq`.
    ## YOUR CODE HERE ##

    # Estimate the complex channel gain coefficients, using the pilot subcarriers.
    H_est = Channel_Estimation(OFDM_freq) # estimate the channel

    # Then equalize -- counteract each subcarrier's channel gain effect on OFDM_freq by
    # dividing it by its estimated channel gain in H_est.
    # Both are vectors and division is element-wise. Call the output `OFDM_est`.

    ## YOUR CODE HERE ##
    OFDM_est = OFDM_freq / H_est

    # The output, at the `dataCarriers` indices are the QPSK symbol values
    # Concatenate them across all OFDM symbols
    OFDM_data = OFDM_est[dataCarriers] # extract the data signal
    if i == 0:
        OFDM_swap =  OFDM_data
    else:
        OFDM_signal = np.concatenate((OFDM_swap,  OFDM_data))
        OFDM_swap = OFDM_signal

# We put QPSK on all subcarriers, so we can merge the constellation plot for all
# received QPSK symbol values, and plot it.
constellation_plot(OFDM_signal)

Finish by running a symbol detector, converting the m-ary symbols to bits, and then converting the bits to a text message. This code prints out the received string.

In [ ]:
# Find the data bits corresponding to these complex symbol values, using the
# 4 possible QPSK symbols
outputVec = np.array([1+1j, -1+1j, 1-1j, -1-1j])
mary_out  = findClosestComplex(OFDM_signal, outputVec)
data_bits  = mary2binary(mary_out, 4)[0]

# Convert the output bits to a character string.
rx_message = binvector2str(data_bits)

🛑  Answer these two questions:
 1. Compute the spectral efficiency, defined as the bit rate (in bits per second) divided by the bandwidth (here, 2 MHz). Note the sampling rate is also 2 MHz, so you can calculate the packet duration.
 2. What is most surprising to you about the OFDM receiver implementation?


### YOUR ANSWER HERE.

🛑 Turn in this notebook on Canvas as a .ipynb file or print to a pdf.